Preprocess the data Peter provided to put it into a ready to analyze form.  save it to the file noted below

In [1]:
output_filename = 'data/yields.parquet'

In [2]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

In [3]:
parquet_file = '~/Downloads/merged_yield_df_area_planeted_harvested_winter_wheat_marked.parquet'
raw0 = pq.read_table(parquet_file).to_pandas()
raw0['Acres Harvested'] = raw0['Acres Harvested'].str.replace(',', '').astype(float)
raw0['Acres Planted'] = raw0['Acres Planted'].str.replace(',', '').astype(float)

#rename columns called County as CountyName
raw0.rename(columns={'County':'CountyName'}, inplace=True)
# new column called County that is the concatenation of State and CountyName
raw0['County'] = raw0['State'] + '/' + raw0['CountyName']
# drop the columns State and CountyName
raw0.drop(columns=['State','CountyName'], inplace=True)
# drop county_name, county_number, state_county_code, state_code
raw0.drop(columns=['qa.landsat','x','y','county_name','county_number','state_county_code','state_code'], inplace=True)



print('raw0: {:,} rows'.format(len(raw0)))


raw0: 24,395,812 rows


In [4]:
# for each year, keep only rows where winter wheat is TRUE
l = []
for y in range(2013,2023):
    mask = (raw0['year'] == y) & (raw0['is_winter_wheat_'+str(y)])
    l.append(raw0[mask])
raw1 = pd.concat(l).sort_values('time')
raw1 = raw1.drop(columns=[c for c in raw1.columns if 'is_winter_wheat' in c])
print('raw1: {:,} rows'.format(len(raw1)))

raw1: 24,395,812 rows


In [5]:
drop_list = ['lat_x', 'lon_x', 'ndvi.sentinel2','qa.sentinel2','lat_y','lon_y','point']
raw2 = raw1.drop(columns=drop_list)

In [6]:
raw3 = raw2.drop(raw2[raw2['ndvi.landsat'] > 1.0].index)
print('raw3: {:,} rows'.format(len(raw3)))
raw4 = raw3.dropna(subset=['ndvi.landsat'])
print('raw4: {:,} rows'.format(len(raw4)))
raw5 = raw4.dropna(subset=['Yield (Bu/Acre)'])
print('raw5: {:,} rows'.format(len(raw5)))

raw3: 24,392,289 rows


KeyboardInterrupt: 

In [ ]:
raw5.head()

In [ ]:

raw6 = raw5[['time','year', 'County','ndvi.landsat', 'Yield (Bu/Acre)','Acres Planted', 'Acres Harvested']]
raw6.head()

In [ ]:
# save raw6 to disk
raw6.to_parquet(output_filename)

In [ ]:
# print the number of unique Counties by year
for y in range(2013,2023):
    mask = (raw6['year'] == y)
    print('Number of unique counties in {}: {:,}'.format(y, len(raw6[mask]['County'].unique())))
    
print('Number of unique counties: {:,}'.format(len(raw6['County'].unique())))